In [8]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings
from terminaltables import AsciiTable
import pandas as pd
import pymysql
import pyquery

warnings.filterwarnings('ignore')
%matplotlib inline

def db_connection(host_name = "ds"):
    host_url = "db.ds.mycelebs.com"
    user_nm = 'dbteam'
    passwd = ''
    port_num = 3306
    db_name = 'book_kyobo'
    conn = pymysql.connect(host=host_url, user = user_nm, passwd=passwd, port=port_num,
                           charset='utf8', db=db_name, cursorclass=pymysql.cursors.DictCursor)
    book_sql = '''SELECT series_id, title, isbn13, listprice, form_size, page, weight, category_name
                  FROM book_kyobo_humanities_data
                  UNION ALL
                  SELECT series_id, title, isbn13, listprice, form_size, page, weight, category_name
                  FROM book_kyobo_learning_data
                  UNION ALL
                  SELECT series_id, title, isbn13, listprice, form_size, page, weight, category_name
                  FROM book_kyobo_lifestyle_data
                  UNION ALL
                  SELECT series_id, title, isbn13, listprice, form_size, page, weight, category_name
                  FROM book_kyobo_literature_data
                  UNION ALL
                  SELECT series_id, title, isbn13, listprice, form_size, page, weight, category_name
                  FROM book_kyobo_science_data
                  UNION ALL
                  SELECT series_id, title, isbn13, listprice, form_size, page, weight, category_name
                  FROM book_kyobo_biz_econo_self_improve_data
                '''
    
    cursor = conn.cursor()
    cursor.execute(book_sql)
    crawl_list = pd.DataFrame(cursor.fetchall()).reset_index()

    return crawl_list

df = db_connection()

# from csv due to db error
# df = pd.read_csv("kyobo_data.csv")
# df = df.drop(['Unnamed: 0', 'index'], axis = 1)
# df['isbn13'].fillna(0, inplace = True)
# df['isbn13'] = df['isbn13'].astype(int)
df.head()

,index,series_id,title,isbn13,listprice,form_size,page,weight,category_name
0,0,9000428,역사를 바꾼 이인자들,1400000010648,11800,153*225mm,316,0,한국역사인물
1,1,9000483,삼국유사(세계의사상 1),1400000212776,17600,148*210mm,508,0,삼국시대일반
2,2,9000484,세계를 움직인 12인의 천재들,1400000212783,13800,153*225mm,356,0,세계역사인물
3,3,9000495,미국역사의 순환,1400000212967,20500,148*210mm,580,0,미국사
4,4,9000596,한국의 멋을 찾아서,1400000215036,19600,188*254mm,437,0,역사기행


# 결측치 확인

In [47]:
# 결측치 확인
for i in df.columns:
    print(f"{i} : {df[i].isnull().sum()}")

index : 0
series_id : 0
title : 0
isbn13 : 0
listprice : 0
form_size : 1137140
page : 334625
weight : 1798410
category_name : 0


# form size, weight, page_num --> 0

In [40]:
# form size 스페이스값을 0으로 변환
df['form_size'].replace(' ', np.nan, inplace = True)

In [41]:
# 무게가 50그람 미만일 경우, 0으로 변환
df.loc[df['weight'] < 50, 'weight'] = np.nan

In [42]:
# 페이지가 10페이지 미만일 경우, 0으로 변환
df.loc[df['page'] < 10, 'page'] = np.nan

# 데이터프레임 복사하기

In [72]:
df_copy = df.copy()

# 무게 당 가격

In [104]:
# 무게 당 가격
df_copy['price_per_100g'] = df_copy['listprice'] / df_copy['weight'] * 100

# 책 넓이

In [71]:
import math

def get_area(x):
    if type(x) == str:
        x = x.split('mm')[0]
        x = x.split('*')
        x = list(map(int, x))
        x = round(math.prod(x))
    else:
        return np.nan
    return x

df['size'] = df['form_size'].apply(get_area)

# 넓이 당 가격 (넓이 : 10mm ^ 2)

In [73]:
# 넓이 당 가격 (넓이 100 mm ^ 2)
df_copy['price_per_100mm^2'] = df_copy['listprice'] / df_copy['size'] * 100

# 페이지 당 가격

In [74]:
# 페이지 당 가격
df_copy['price_per_page'] = df_copy['listprice'] / df_copy['page']

# 그림이 많을 것 같은 책 구분자 표시

In [77]:
for i in df_copy['category_name'].unique():
    print(i)

한국역사인물
삼국시대일반
세계역사인물
미국사
역사기행
문화일반
한국사
신화
세계사
역사일반
동양사
역사/문화
역사인물
문화사
해방전후사/한국전쟁
독도문제
가정의례
풍속/민속학일반
교양세계사
민족운동/민족주의
한국통사
일본사일반
한국고대사
동양사일반
한국문화사
일제시대
중국사일반
한국근현대사
성풍속사
고고학
일본문화사
문화사일반
세계문화사
민속학
조선시대일반
무속/민간신앙
역사이론
러시아사
중국근대사
조선정치사
한국현대사
동남아시아사
서양문화사
한국지리
북한사
고려시대
신라
지리학의 이해
조선후기
고구려
한일관계와역사
문명/문명사
삼국사기
중국역사인물
전설/설화
지리학사/사전
신화의이해
삼국유사
한국역사기행
조선전기
지방/향토
민족학
인류학
서양사일반
발해
성씨/족보
한국근대사
그리스로마신화
조선중기
독일사
로마사
그리스
통일신라
유럽사일반
미국역사인물
청소년 역사
유럽역사인물
중국신화
한국신화
지리이야기
탐험사
민속놀이
조선왕조사
아프리카사
일본근현대사
백제
박물관학
문화재/유산
근대개화기
생활사
음식문화
문화유적답사
이스라엘/유대인
가정윤리/효
영국사
프랑스사
의복문화
역사이야기
기타나라사
역사철학
유적/유물
몽골/티벳
이탈리아사
인도사
조선시대생활/문화
중국문화사
중국근세사
세계사이야기
이슬람사
일본역사인물
세계지리
라틴아메리카사
동양신화
유럽신화
중국고대사
역사기행일반
동양문화사
기타아시아역사인물
중국근현대사
중국중세사
주거문화
대만/마카오/홍콩사
일본고대사
군사정권과민주화운동
유럽문화사
중앙아시아사
캐나다/호주사

미국문화사
일본신화
이집트사
중국역사기행
기타국가역사인물
이집트역사기행
서양신화
중동/기타
유럽역사기행
조선이야기
기타역사기행
대학교재
건국사/멸망사
일본중세사
일본역사기행
지식/상식
친일파
인문교양
지중해/그리스여행
인문
인문교양총서일반
국어이야기
공자
중국철학
고사성어
문학이론일반
문법론
사기열전
한국문학이론일반
기타사상가
장폴사르트르
인문학일반
한국고전문학
한문학
주역
철학
영미문학론
국어문법론
심리학
언어학
역학/사주
인문교양총서
독서

결혼
이혼
육아일기
조기영어교육
수교육
태교동화
홈인테리어일반
유태인교육법
가베
임산부건강
모유수유
경제교육
기공체조
건강일반
한방치료
뇌건강
질병치료/예방
건강이야기
건강
다이어트
건강문고
다이어트일반
피부관리방법
마사지
단학
암
다이어트비디오
수맥
식품과건강
명상
자연치유
자연식건강
민간요법
손/발건강
장수
건강상식
한방건강
성생활
금연/금주
스트레스
성인병
기건강
헬스/웨이트트레이닝
동의보감
단전호흡
경락/경혈
응급처치
대체의학
물건강
자연요법일반
한방약초
국선도
뇌혁명
수험생건강법
수면
요가
건강음식
사상체질
피부미용일반
치질/변비
단식
스트레칭
여성건강
최면치료
허브/아로마요법
자연식품/생식
뇌호흡
약물중독
디스크/척추
노후건강관리
요통
아토피/여드름(피부질환)
눈건강
식생활과건강
정신건강
남성건강
치매
코/귀/치아건강
탈모
잘 먹고 잘사는 법
새집증후군
FAMILY DOCTOR
건강문고기타
로맨스만화
웹툰/카툰에세이
기타만화
액션/무협만화
역사만화
명랑/코믹만화
탐정/추리
직업만화
성인만화
SF/판타지
일상생활/드라마/가족만화
스포츠만화
고전/문학작품만화
요리만화
동물등장만화
공포/스릴러
학원만화
웹툰일반
BL만화
도박만화
시사/비평
그래픽노블일반
교양
일상생활/드라마
액션/무협
캐릭터퍼즐
캐릭터일반
로맨스
명랑/코믹
실용/취미
세계여행
국내여행
국내여행일반
해외여행
여행에세이
테마여행
여행
도로지도
전국지도
유럽여행일반
중국여행
세계지도
답사여행
호주/뉴질랜드
아프리카여행
아시아여행일반
미국여행
동남아여행
유럽지도
아프리카지도
서울/수도권
캠핑/드라이브/기차여행
경상도
홍콩/마카오/대만여행
일본여행
충청도
배낭여행
제주도
괌/사이판/하와이
강원도
전라도
오지탐험/산여행
인도/티베트여행
맛집여행
지구본
아시아지도
북남미지도
주말여행
터키여행
프랑스여행
이탈리아여행
체험/미술관/박물관여행
바다/섬여행
워킹홀리데이
실크로드여행
영국여행
사찰여행
스페인여행
카페여행
중남미여행
지도
해외지도
국내지도
북남미여행
유럽여행
러시아여행
독일여행
캐나다여행
뉴욕


In [78]:
# 문자 수 10%
child_list = ['다이어리/플래너', '메모/문서서식', '토이북', '유아학습', '유아그림책,유아(0~7세)', '스티커북/색칠하기',
              '고전/전래동화/명작동화', '동시/동요', '어린이작가', '그림책', '동화책', '어린이기독교', '동화/그림책/리더스북',
              '영어만화', '어린이중국어', '명작동화', '명작동화', '전래동화', '어린이전집', '어린이', '유아완구/교구/블록',
              '세계명작그림책', '그림찾기/미로/게임북', '동요/동시']

# 문자 수 20%
ele_list = ['그림에세이', '초등2학년', '초등5학년', '예체능', '초등1학년', '초등3학년', '초등4학년', '아이디어', '유머',
            '시조/한시(고전)', '기타나라/여러나라시', '사진이 있는 시', '시가', '포토에세이', '한시', '사랑시', '일본시',
            '그림이 있는 시', '영미시', '중국시', '독일시', '명시모음','프랑스시', '나라별 시', '시/에세이문고', '기타나라시']

df_copy['check_drawing_10'] = 0
df_copy['check_drawing_20'] = 0

def drawing_10(x):
    if x in child_list:
        return True
    else:
        return False
    
def drawing_20(x):
    if x in ele_list:
        return True
    else:
        return False
    
df_copy['check_drawing_10'] = df_copy['category_name'].apply(drawing_10)
df_copy['check_drawing_20'] = df_copy['category_name'].apply(drawing_20)

# 글자수 데이터가 있는 테이블 가져오기

In [79]:
# 글자수 데이터 추출

def db_connection(host_name = "ds"):
    host_url = "db.ds.mycelebs.com"
    user_nm = 'dbteam'
    passwd = 'celebdbteam2@#'
    port_num = 3306
    db_name = 'book_kyobo'
    conn = pymysql.connect(host=host_url, user = user_nm, passwd=passwd, port=port_num,
                           charset='utf8', db=db_name, cursorclass=pymysql.cursors.DictCursor)
    book_sql = '''SELECT isbn, text_len
                  FROM book_kyobo_matched_data
                '''
    
    cursor = conn.cursor()
    cursor.execute(book_sql)
    crawl_list = pd.DataFrame(cursor.fetchall()).reset_index()

    return crawl_list

df_2 = db_connection()
df_2.rename(columns = {'isbn':'isbn13'}, inplace = True)
df_2.head()

,index,isbn13,text_len
0,0,9788901150598,약 34만 자
1,1,9788901152295,약 10.3만 자
2,2,9788901166285,약 11.7만 자
3,3,9788901217666,약 10.8만 자
4,4,9788925556826,약 49.1만 자


# 글자수 데이터가 있는 행의 숫자 추출

In [80]:
import re
def extract(x):
    num = float(''.join(ele for ele in x if ele.isdigit() or ele == '.'))
    return num * 10000

df_2['text_len'] = df_2['text_len'].apply(extract)

# 글자수 데이터 있는 테이블과 원본 데이터프레임 Merge

In [81]:
df_copy['isbn13'] = df_copy['isbn13'].astype(int)

In [82]:
df_2['isbn13'] = df_2['isbn13'].astype(int)

In [138]:
df_big = df_copy.merge(df_2, how = 'left', on = 'isbn13')

In [139]:
df_big.drop(columns = ['index_x', 'index_y'], inplace = True)

# 글자수 결측값 채워주기
- 100mm^2 당 평균 2글자
- size 컬럼 / 100 * 2

In [140]:
df_big.loc[df_big['text_len'].isnull(), 'text_len'] = round(df_big['size'] / 100 * 2 * df_big['page'], 2)

In [141]:
df_big.loc[df_big['check_drawing_10'] == True, 'text_len'] = df_big['text_len'] * 0.1
df_big.loc[df_big['check_drawing_20'] == True, 'text_len'] = df_big['text_len'] * 0.2

# 사이즈 대비 무게
- 사이즈에 비해 무게가 가벼울수록 들고 다니기 편안함.
- 해당 컬럼 값이 작을수록 좋다는 뜻.

In [142]:
df_big['size_per_weight'] = round(df_big['weight'] / df_big['size'], 2)

# 가격 대비 글자수
- 가격에 비해 글자수가 많다면 더 많은 내용이 담겨있는것이고, 가성비가 좋다고 할 수 있음.

In [143]:
df_big['price_per_text_len'] = round(df_big['text_len'] / df_big['listprice'], 2)

# inf 값 수정

In [144]:
li_inf = ['price_per_100g', 'price_per_100mm^2', 'price_per_page', 'size_per_weight', 'price_per_text_len']

for i in li_inf:
    df_big[i] = df_big[i].replace(np.inf, np.nan)

# 0인 값 nan으로 전환

In [145]:
li_nan = ['price_per_100g', 'price_per_100mm^2', 'price_per_page', 'size_per_weight', 'price_per_text_len']

for i in li_nan:
    df_big.loc[df_big[i] == 0, i] = np.nan

In [146]:
df_big.head()

,series_id,title,isbn13,listprice,form_size,page,weight,category_name,size,price_per_100mm^2,price_per_page,check_drawing_10,check_drawing_20,price_per_100g,text_len,size_per_weight,price_per_text_len
0,9000428,역사를 바꾼 이인자들,1400000010648,11800,153*225mm,316.0,NaN,한국역사인물,34425.0,34.277415,37.341772,False,False,NaN,217566.00,NaN,18.44
1,9000483,삼국유사(세계의사상 1),1400000212776,17600,148*210mm,508.0,NaN,삼국시대일반,31080.0,56.628057,34.645669,False,False,NaN,315772.80,NaN,17.94
2,9000484,세계를 움직인 12인의 천재들,1400000212783,13800,153*225mm,356.0,NaN,세계역사인물,34425.0,40.087146,38.764045,False,False,NaN,245106.00,NaN,17.76
3,9000495,미국역사의 순환,1400000212967,20500,148*210mm,580.0,NaN,미국사,31080.0,65.958816,35.344828,False,False,NaN,360528.00,NaN,17.59
4,9000596,한국의 멋을 찾아서,1400000215036,19600,188*254mm,437.0,NaN,역사기행,47752.0,41.045401,44.851259,False,False,NaN,417352.48,NaN,21.29


In [147]:
li_qcut = ['price_per_100g', 'price_per_100mm^2', 'price_per_page', 'size_per_weight', 'price_per_text_len']


for i in li_qcut:
    sample_df = df_big.copy()
    print(f"{i} : {pd.qcut(sample_df[i], 5, duplicates = 'drop')}")

price_per_100g : 0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
          ... 
2262886    NaN
2262887    NaN
2262888    NaN
2262889    NaN
2262890    NaN
Name: price_per_100g, Length: 2262891, dtype: category
Categories (5, interval[float64]): [(1.8900000000000001, 2222.222] < (2222.222, 2710.414] < (2710.414, 3157.895] < (3157.895, 3867.403] < (3867.403, 1440000.0]]
price_per_100mm^2 : 0               (32.069, 40.85]
1          (54.873, 51200000.0]
2               (32.069, 40.85]
3          (54.873, 51200000.0]
4               (40.85, 54.873]
                   ...         
2262886         (40.85, 54.873]
2262887         (40.85, 54.873]
2262888     (-0.000429, 20.914]
2262889         (40.85, 54.873]
2262890         (40.85, 54.873]
Name: price_per_100mm^2, Length: 2262891, dtype: category
Categories (5, interval[float64]): [(-0.000429, 20.914] < (20.914, 32.069] < (32.069, 40.85] < (40.85, 54.873] < (54.873, 51200000.0]]
price_per_page : 0              (26.8

In [148]:
li_cat = ['cat_price_per_100g', 'cat_price_per_100mm^2', 'cat_price_per_page', 'cat_size_per_weight', 'cat_price_per_text_len']

for i in li_cat:
    df_big[i] = 0

df_big.loc[(df_big['price_per_100g'] <= 2222.222) & (df_big['price_per_100g'] > 1.8900000000000001), 'cat_price_per_100g'] = 1
df_big.loc[(df_big['price_per_100g'] <= 2710.414) & (df_big['price_per_100g'] > 2222.222), 'cat_price_per_100g'] = 2
df_big.loc[(df_big['price_per_100g'] <= 3157.895) & (df_big['price_per_100g'] > 2710.414), 'cat_price_per_100g'] = 3
df_big.loc[(df_big['price_per_100g'] <= 3867.403) & (df_big['price_per_100g'] > 3157.895), 'cat_price_per_100g'] = 4
df_big.loc[(df_big['price_per_100g'] <= 1440000.0) & (df_big['price_per_100g'] > 3867.403), 'cat_price_per_100g'] = 5

df_big.loc[(df_big['price_per_100mm^2'] <= 20.914) & (df_big['price_per_100mm^2'] > -0.000429), 'cat_price_per_100mm^2'] = 1
df_big.loc[(df_big['price_per_100mm^2'] <= 32.069) & (df_big['price_per_100mm^2'] > 20.914), 'cat_price_per_100mm^2'] = 2
df_big.loc[(df_big['price_per_100mm^2'] <= 40.85) & (df_big['price_per_100mm^2'] > 32.069), 'cat_price_per_100mm^2'] = 3
df_big.loc[(df_big['price_per_100mm^2'] <= 54.873) & (df_big['price_per_100mm^2'] > 40.85), 'cat_price_per_100mm^2'] = 4
df_big.loc[(df_big['price_per_100mm^2'] <= 51200000.0) & (df_big['price_per_100mm^2'] > 54.873), 'cat_price_per_100mm^2'] = 5

df_big.loc[(df_big['price_per_page'] <= 26.882) & (df_big['price_per_page'] > 0.004), 'cat_price_per_page'] = 1
df_big.loc[(df_big['price_per_page'] <= 40.625) & (df_big['price_per_page'] > 26.882), 'cat_price_per_page'] = 2
df_big.loc[(df_big['price_per_page'] <= 53.398) & (df_big['price_per_page'] > 40.625), 'cat_price_per_page'] = 3
df_big.loc[(df_big['price_per_page'] <= 79.365) & (df_big['price_per_page'] > 53.398), 'cat_price_per_page'] = 4
df_big.loc[(df_big['price_per_page'] <= 281936.667) & (df_big['price_per_page'] > 79.365), 'cat_price_per_page'] = 5

df_big.loc[(df_big['size_per_weight'] <= 0.02) & (df_big['size_per_weight'] > 0.009000000000000001), 'cat_size_per_weight'] = 1
df_big.loc[(df_big['size_per_weight'] <= 334.8) & (df_big['size_per_weight'] > 0.02), 'cat_size_per_weight'] = 5

df_big.loc[(df_big['price_per_text_len'] <= 8.6) & (df_big['price_per_text_len'] > 0.009000000000000001), 'cat_price_per_text_len'] = 1
df_big.loc[(df_big['price_per_text_len'] <= 12.72) & (df_big['price_per_text_len'] > 8.6), 'cat_price_per_text_len'] = 2
df_big.loc[(df_big['price_per_text_len'] <= 16.51) & (df_big['price_per_text_len'] > 12.72), 'cat_price_per_text_len'] = 3
df_big.loc[(df_big['price_per_text_len'] <= 20.8) & (df_big['price_per_text_len'] > 16.51), 'cat_price_per_text_len'] = 4
df_big.loc[(df_big['price_per_text_len'] <= 1401973.76) & (df_big['price_per_text_len'] > 20.8), 'cat_price_per_text_len'] = 5

In [149]:
li_cat = ['cat_price_per_100g', 'cat_price_per_100mm^2', 'cat_price_per_page', 'cat_size_per_weight', 'cat_price_per_text_len']

for i in li_cat:
    df_big.loc[df_big[i] == 0, i] = np.nan

# 중간 실행 위치!!!!!

In [175]:
df_3 = df_big.copy()

# carriage_ord

In [177]:
df_3['carriage_ord'] = 0

df_3.loc[((df_3['size'] >= 124740.0) | (df_3['weight'] > 980.0)), 'carriage_ord'] = 1

0    2208569
1      54322
Name: carriage_ord, dtype: int64

In [178]:
pd.qcut(df_3.loc[df_3['carriage_ord'] == 0]['size_per_weight'], 5, duplicates = 'drop')

0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
          ... 
2262885    NaN
2262886    NaN
2262887    NaN
2262889    NaN
2262890    NaN
Name: size_per_weight, Length: 2208569, dtype: category
Categories (2, interval[float64]): [(0.009000000000000001, 0.02] < (0.02, 250.0]]

In [182]:
df_3.loc[(df_3['size_per_weight'] <= 0.02) & (df_3['size_per_weight'] > 0.009000000000000001), 'carriage_ord'] = 3
df_3.loc[(df_3['size_per_weight'] <= 250.0) & (df_3['size_per_weight'] > 0.02), 'carriage_ord'] = 2

In [181]:
df_3.tail(5)

,series_id,title,isbn13,listprice,form_size,page,weight,category_name,size,price_per_100mm^2,...,price_per_100g,text_len,size_per_weight,price_per_text_len,cat_price_per_100g,cat_price_per_100mm^2,cat_price_per_page,cat_size_per_weight,cat_price_per_text_len,carriage_ord
2262886,46195621,17년째 버티고 있는 중입니다(셀프헬프 시리즈 16),9788965292647,12800,128*188mm,152.00,nan,자기계발,24064.00,53.19,...,nan,73154.56,nan,5.72,nan,4.00,5.00,nan,1.00,0
2262887,46195665,코로나 시대 자영업의 미래,9791197076817,17800,147*225mm,384.00,nan,경제/경영,33075.00,53.82,...,nan,254016.00,nan,14.27,nan,4.00,3.00,nan,3.00,0
2262888,46195669,주식 공부 5일 완성(마흔 살에 시작하는)(개정판),9791162541869,18000,152*2250mm,336.00,nan,경제/경영,342000.00,5.26,...,nan,2298240.00,nan,127.68,nan,1.00,4.00,nan,5.00,1
2262889,46195670,파워링커 혁명,9791129019646,16000,152*225mm,257.00,nan,자기계발,34200.00,46.78,...,nan,175788.00,nan,10.99,nan,4.00,4.00,nan,2.00,0
2262890,46195672,더 위험한 미국이 온다,9791162541883,16000,152*225mm,276.00,nan,경제/경영,34200.00,46.78,...,nan,188784.00,nan,11.80,nan,4.00,4.00,nan,2.00,0


In [162]:
div_1 = df_big.iloc[:1000000]
div_2 = df_big.iloc[1000000:2000000]
div_3 = df_big.iloc[2000000:]

div_1.to_csv("output_div_1.csv", encoding = 'utf-8-sig')
div_2.to_csv("output_div_2.csv", encoding = 'utf-8-sig')
div_3.to_csv("output_div_3.csv", encoding = 'utf-8-sig')

In [165]:
df_big.columns

Index(['series_id', 'title', 'isbn13', 'listprice', 'form_size', 'page',
       'weight', 'category_name', 'size', 'price_per_100mm^2',
       'price_per_page', 'check_drawing_10', 'check_drawing_20',
       'price_per_100g', 'text_len', 'size_per_weight', 'price_per_text_len',
       'cat_price_per_100g', 'cat_price_per_100mm^2', 'cat_price_per_page',
       'cat_size_per_weight', 'cat_price_per_text_len'],
      dtype='object')

In [159]:
# df_big.to_csv('output.csv', encoding = 'utf-8-sig')